#### 讀取檔案並秀出

In [1]:
from OCC.Display.SimpleGui import init_display
from OCC.Extend.DataExchange import read_iges_file

shapes = read_iges_file(r"C:\alan\ITRI\model\cube.IGS")
# 初始化 3D 顯示環境
display, start_display, add_menu, add_function_to_menu = init_display()
display.DisplayShape(shapes, update=True)
start_display()

#### 將物件中所有面都畫出來

In [2]:
import time
from OCC.Display.SimpleGui import init_display
from OCC.Core.IGESControl import IGESControl_Reader
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopoDS import topods_Face
from OCC.Core.TopAbs import TopAbs_FACE

# 初始化 3D 顯示環境
display, start_display, add_menu, add_function_to_menu = init_display()

# IGS檔案路徑
file_path = r"C:\alan\ITRI\model\cylinder.IGS"

# 讀取檔案
reader = IGESControl_Reader()
status = reader.ReadFile(file_path)
if status == 1:  # 確認文件是否存在
    reader.TransferRoots()
    shape = reader.Shape()

    # 找所有的面
    explorer = TopExp_Explorer(shape, TopAbs_FACE)
    while explorer.More():
        face = topods_Face(explorer.Current())

        # 清除當前顯示
        display.EraseAll()

        # 顯示目前的面
        display.DisplayShape(face, update=True)
        display.FitAll()

        time.sleep(2)  # 暫停兩秒

        # 移動到下一個面
        explorer.Next()
else:
    print("Failed to read the file.")

# 啟動 OCC 視窗
start_display()

C:\Users\Martin\AppData\Local\Temp\ipykernel_9272\3441983799.py:24: DeprecationWarning: Call to deprecated function topods_Face since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method topods.Face
  face = topods_Face(explorer.Current())


#### 將物件中所有面畫出來並儲存

In [3]:
import os
import time
from OCC.Display.SimpleGui import init_display
from OCC.Core.IGESControl import IGESControl_Reader
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopoDS import topods_Face
from OCC.Core.TopAbs import TopAbs_FACE

display, start_display, add_menu, add_function_to_menu = init_display()

file_path = r"C:\alan\ITRI\model\cube.IGS"

# 影像的保存路徑
image_directory = r"C:\alan\ITRI\images"
if not os.path.exists(image_directory):
    os.makedirs(image_directory)


reader = IGESControl_Reader()
status = reader.ReadFile(file_path)
if status == 1:  
    reader.TransferRoots()
    shape = reader.Shape()

    index = 1
    explorer = TopExp_Explorer(shape, TopAbs_FACE)
    while explorer.More():
        face = topods_Face(explorer.Current())

        display.EraseAll()

        display.DisplayShape(face, update=True, color='BLUE', transparency=0.5)  # 可以調整顏色和透明度
        display.FitAll()

        time.sleep(2)  # 暫停兩秒

        # 儲存每個面的影像
        image_path = os.path.join(image_directory, f'face_{index}.png')
        display.View.Dump(image_path)
        print(f'Saved image of face {index} to {image_path}')

        # 移動到下一個面
        index += 1
        explorer.Next()
else:
    print("Failed to read the file.")
start_display()

C:\Users\Martin\AppData\Local\Temp\ipykernel_9272\1804153163.py:28: DeprecationWarning: Call to deprecated function topods_Face since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method topods.Face
  face = topods_Face(explorer.Current())


Many colors for color name BLUE, using first.
Saved image of face 1 to C:\alan\ITRI\images\face_1.png


#### 去計算每個面的一些資訊(面積，上下界，面的類型)

In [4]:
import os
import time
from OCC.Display.SimpleGui import init_display
from OCC.Core.IGESControl import IGESControl_Reader
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopoDS import topods_Face
from OCC.Core.TopAbs import TopAbs_FACE
from OCC.Core.BRepTools import breptools_UVBounds
from OCC.Core.GeomLProp import GeomLProp_SLProps
from OCC.Core.BRepGProp import brepgprop_SurfaceProperties
from OCC.Core.BRepAdaptor import BRepAdaptor_Surface
from OCC.Core.GProp import GProp_GProps
from OCC.Core.gp import gp_Pnt

file_path = r"C:\alan\ITRI\model\cylinder.IGS"

reader = IGESControl_Reader()
status = reader.ReadFile(file_path)
if status == 1: 
    reader.TransferRoots()
    shape = reader.Shape()

    index = 1
    explorer = TopExp_Explorer(shape, TopAbs_FACE)
    while explorer.More():
        face = topods_Face(explorer.Current())
        
        adaptor_surface = BRepAdaptor_Surface(face, True)
        surface_type = adaptor_surface.GetType()              # 獲取面的類型
    
        # 獲取面的邊界
        u_min, u_max, v_min, v_max = adaptor_surface.FirstUParameter(), adaptor_surface.LastUParameter(), adaptor_surface.FirstVParameter(), adaptor_surface.LastVParameter()
    
        # 計算面積
        gprops = GProp_GProps()                             # 用於獲取面的質量等信息
        brepgprop_SurfaceProperties(face, gprops)           # 計算面的面積
        area = gprops.Mass()                                # 獲取面的面積

        print(f"Face {index}:")
        print(f"  Type: {surface_type}")
        print(f"  Bounds in U: {u_min} to {u_max}")
        print(f"  Bounds in V: {v_min} to {v_max}")
        print(f"  Area: {area}")

        index += 1
        explorer.Next()

Face 1:
  Type: 7
  Bounds in U: 1.5707963271795888 to 4.712388980179589
  Bounds in V: -1.0463605493570475e-13 to 99.99999999999994
  Area: 17703.274620218293


C:\Users\Martin\AppData\Local\Temp\ipykernel_9272\2229307294.py:26: DeprecationWarning: Call to deprecated function topods_Face since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method topods.Face
  face = topods_Face(explorer.Current())
C:\Users\Martin\AppData\Local\Temp\ipykernel_9272\2229307294.py:36: DeprecationWarning: Call to deprecated function brepgprop_SurfaceProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.SurfaceProperties
  brepgprop_SurfaceProperties(face, gprops)           # 計算面的面積


#### 讀取檔案後計算每個面的法向量 以及 最大跟最小曲率

In [5]:
import os
from OCC.Core.IGESControl import IGESControl_Reader
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopoDS import topods_Face
from OCC.Core.BRepAdaptor import BRepAdaptor_Surface
from OCC.Core.BRepLProp import BRepLProp_SLProps
from OCC.Core.TopAbs import TopAbs_FACE
from OCC.Core.gp import gp_Pnt2d

# IGS檔案路徑
file_path = r"C:\alan\ITRI\model\cylinder.IGS"

# 確認文件是否存在
if not os.path.exists(file_path):
    print("檔案路徑錯誤！")
else:
    # 讀取檔案
    reader = IGESControl_Reader()
    reader.ReadFile(file_path)
    reader.TransferRoots()          # 使用 TransferRoots 方法轉換模型數據
    shape = reader.Shape()          # 獲取模型的形狀

    # 找所有的面
    explorer = TopExp_Explorer(shape, TopAbs_FACE)
    while explorer.More():
        face = topods_Face(explorer.Current())               # 獲取當前的面 explorer.Current()
        adaptor_surface = BRepAdaptor_Surface(face, True)    # 創建一個物件用於分析面的幾何特性
        props = BRepLProp_SLProps(adaptor_surface, 2, 0.01)  # 用於獲取面的曲率信息，2 表示計算主曲率，也就是曲面在特定點的最大和最小曲率，0.01控制計算精度
        u_min, u_max, v_min, v_max = adaptor_surface.FirstUParameter(), adaptor_surface.LastUParameter(), adaptor_surface.FirstVParameter(), adaptor_surface.LastVParameter()
        u_mid = (u_min + u_max) / 2 
        v_mid = (v_min + v_max) / 2                          # 計算面的中心點坐標
        props.SetParameters(u_mid, v_mid)                    # 設置曲率計算的參數，到時候求出法向量就為該點的向量
        if props.IsCurvatureDefined():                       # 檢查曲率是否被定義
            max_curvature = props.MaxCurvature()             # 最大曲率
            min_curvature = props.MinCurvature()             # 最小曲率
            print("最大曲率:", max_curvature)
            print("最小曲率:", min_curvature)
            normal = props.Normal()                          # 計算該點的法向量
            # 如果面朝向是反的，反轉法向量
            if face.Orientation() == TopAbs_FACE:            # 檢查面的朝向是否為反向
                normal.Reverse()
            print("法向量:", normal.X(), normal.Y(), normal.Z())
        explorer.Next()                                      # 進入下一個面

最大曲率: 0.017745827935200725
最小曲率: 0.0
法向量: -8.979205123174363e-11 0.0 -1.0


C:\Users\Martin\AppData\Local\Temp\ipykernel_9272\1712942013.py:26: DeprecationWarning: Call to deprecated function topods_Face since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method topods.Face
  face = topods_Face(explorer.Current())               # 獲取當前的面 explorer.Current()


In [6]:
import os
from OCC.Core.IGESControl import IGESControl_Reader
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopoDS import topods_Face
from OCC.Core.BRepAdaptor import BRepAdaptor_Surface
from OCC.Core.BRepLProp import BRepLProp_SLProps
from OCC.Core.TopAbs import TopAbs_FACE
from OCC.Core.gp import gp_Pnt2d, gp_Dir

# IGS檔案路徑
file_path = r"C:\alan\ITRI\model\cylinder.IGS"

# 確認文件是否存在
if not os.path.exists(file_path):
    print("檔案路徑錯誤！")
else:
    # 讀取檔案
    reader = IGESControl_Reader()
    reader.ReadFile(file_path)
    reader.TransferRoots()          # 使用 TransferRoots 方法轉換模型數據
    shape = reader.Shape()          # 獲取模型的形狀

    # 找所有的面
    explorer = TopExp_Explorer(shape, TopAbs_FACE)
    while explorer.More():
        face = topods_Face(explorer.Current())               # 獲取當前的面 explorer.Current()
        adaptor_surface = BRepAdaptor_Surface(face, True)    # 創建一個物件用於分析面的幾何特性
        props = BRepLProp_SLProps(adaptor_surface, 2, 0.01)  # 用於獲取面的曲率信息，2 表示計算主曲率，也就是曲面在特定點的最大和最小曲率，0.01控制計算精度
        u_min, u_max, v_min, v_max = adaptor_surface.FirstUParameter(), adaptor_surface.LastUParameter(), adaptor_surface.FirstVParameter(), adaptor_surface.LastVParameter()
        u_mid = (u_min + u_max) / 2 
        v_mid = (v_min + v_max) / 2                          # 計算面的中心點坐標
        props.SetParameters(u_mid, v_mid)                    # 設置曲率計算的參數，到時候求出法向量就為該點的向量
        if props.IsCurvatureDefined():                       # 檢查曲率是否被定義
            max_curvature = props.MaxCurvature()             # 最大曲率
            min_curvature = props.MinCurvature()             # 最小曲率
            print("最大曲率:", max_curvature)
            print("最小曲率:", min_curvature)
            max_curvature_dir = props.MaxCurvatureDirection() # 最大曲率方向
            normal = props.Normal()                          # 計算該點的法向量
            # 如果面朝向是反的，反轉法向量
            if face.Orientation() == TopAbs_FACE:            # 檢查面的朝向是否為反向
                normal.Reverse()
            print("法向量:", normal.X(), normal.Y(), normal.Z())
            print("最大曲率方向:", max_curvature_dir.X(), max_curvature_dir.Y(), max_curvature_dir.Z())
        explorer.Next()

C:\Users\Martin\AppData\Local\Temp\ipykernel_9272\2365779602.py:26: DeprecationWarning: Call to deprecated function topods_Face since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method topods.Face
  face = topods_Face(explorer.Current())               # 獲取當前的面 explorer.Current()


最大曲率: 0.017745827935200725
最小曲率: 0.0


AttributeError: 'BRepLProp_SLProps' object has no attribute 'MaxCurvatureDirection'